In [1]:
from estnltk.wordnet import wn
import sqlite3
import igraph as ig
import plotly.plotly as py
from plotly.graph_objs import *

In [2]:
sset_db = 'wordnet/data/all_synsets.db'
relation_db = 'wordnet/data/all_relations_strings.db'

In [3]:
g = ig.Graph()
g.es["relation"] = []
g.vs["synset"] =[]

In [4]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print("Connection error: [%s]" % e)

    return None

def create_table(conn, create_table_sql ):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except sqlite3.Error as e:
        print("Connection error while creating table: [%s]" % e)

def sqlTables(databaseLoc):

    sql_create_synset_table = ''' CREATE TABLE IF NOT EXISTS graph_table(

                                        start_vertex INT NOT NULL,
                                        start_synset TEXT NOT NULL,
                                        end_synset TEXT NOT NULL,
                                        end_vertex INT NOT NULL,
                                        relation TEXT NOT NULL
                                                    ); '''
    conn = create_connection(databaseLoc)
    if conn is not None:
        create_table(conn,sql_create_synset_table)
    else:
        print("Error! cannot create db conn.")

In [5]:
def get_ssets(db_file):

    sqlTables(db_file)
    conn = create_connection(db_file)
    cursor = conn.cursor()
    with conn:
        cursor.execute("SELECT DISTINCT synset_word FROM synset_table")
        for row in cursor.fetchall():
            sset_word.append(row[0])

In [6]:


def get_relations(db_file):
    
    sqlTables(db_file)
    conn = create_connection(db_file)
    cursor = conn.cursor()
    
    with conn:
        cursor.execute("SELECT start_vertex FROM graph_table")
        for row in cursor.fetchall():
            start_vrtx.append(row[0])
        cursor.execute("SELECT end_vertex FROM graph_table")
        for row in cursor.fetchall():
            end_vrtx.append(row[0])
        cursor.execute("SELECT relation FROM graph_table")
        for row in cursor.fetchall():
            relation.append(row[0])

In [7]:
def add_edges(start_vertex, end_vertex, N):
    '''
    Edges are added between start_vertex[i] and end_vertex
    If end vertex ID changes, it means that there are no more start IDs corresponding to end ID, so
    we take N-1 as end ID and continiue adding start_vertex[i] to end_vertex[N-1], etc.
    '''
    sset_list = [None]*N
    rel_list = [None]*N
    #max_vrt = max(max(start_vertex[N]), max(end_vertex[N]))
    g.add_vertices(N)
    end = int((N/2)-1) # edges indexed from 0
    j=1
    print(end)
    for i in range(N):
        g.add_edge(i, end)        
        sset_list[i] = sset_word[ start_vertex[i] ]
        sset_list[N-(i+1)] = sset_word[ end_vertex[i] ]
        #rel_list[i] = relation[i]
        if end_vertex[i] != end_vertex[i+1]:
            end = N-j
            j+=1            
    g.es["relation"] = relation[:N]#rel_list
    g.vs["synset"] = sset_list
    


In [8]:
start_vrtx = []
end_vrtx   = []
relation   = []

get_relations(relation_db)

In [9]:
sset_word = []
get_ssets(sset_db)

In [20]:
sset_word[:20]

['hea',
 'viimane',
 'nooruslik',
 'keskmine',
 'pikakoivaline',
 'kena',
 'kiire',
 'kindel',
 'õige',
 'kääbuslik',
 'hiiglaslik',
 'massiivne',
 'mahukas',
 'ulatuslik',
 'ruumikas',
 'miniatuurne',
 'pisitilluke',
 'mikroskoopiline',
 'uhiuus',
 'halb']

In [66]:
g = ig.Graph()
g.vs["synset"]=[]
g.es["relation"]=[]
amount = 100
add_edges(start_vrtx, end_vrtx, amount)

49


In [64]:
print(len(start_vrtx))
print(len(end_vrtx))
g.vs["synset"]

145889
145889


['matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'matsakas',
 'kahjasinine',
 'kahkjassinine',
 'kahvatusinine',
 'kannikesesinine',
 'kirgassinine',
 'koobaltsinine',
 'krellsinine',
 'külmsinine',
 'lasuursinine',
 'lavendelsinine',
 'lillakassinine',
 'linaõissinine',
 'mahesinine',
 'meresinine',
 'metüleensinine',
 'mustjassinine',
 'mustsinine',
 'piimjassinine',
 'pimestavsinine',
 'pleeksinine',
 'pl

In [73]:
layout = g.layout("kamada_kawai")
ig.plot(g, layout = layout)

AttributeError: 'bytes' object has no attribute 'encode'

In [67]:

labels=list(g.vs['synset'])
N=len(labels)
E=[e.tuple for e in g.es]# list of edges
layt=g.layout('kk') #spherical layout
type(layt)

igraph.layout.Layout

In [68]:
ig.plot(g)

AttributeError: 'bytes' object has no attribute 'encode'

In [70]:
Xn=[layt[k][0] for k in range(N)]
Yn=[layt[k][1] for k in range(N)]
Xe=[]
Ye=[]
for e in E:
    Xe+=[layt[e[0]][0],layt[e[1]][0], None]
    Ye+=[layt[e[0]][1],layt[e[1]][1], None] 
    
trace1=Scatter(x=Xe,
               y=Ye,
               mode='lines',
               line= dict(color='rgb(210,210,210)', width=1),
               hoverinfo='none'
               )
trace2=Scatter(x=Xn,
               y=Yn,
               mode='markers',
               name='ntw',
               marker=dict(symbol='circle-dot',
                                        size=5,
                                        color='#6959CD',
                                        line=dict(color='rgb(50,50,50)', width=0.5)
                                        ),
               text=labels,
               hoverinfo='text'
               )

axis=dict(showline=False, # hide axis line, grid, ticklabels and  title
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title='' 
          )

width=800
height=800
layout=Layout(title= "Coauthorship network of scientists working on network theory and experiment"+\
              "<br> Data source: <a href='https://networkdata.ics.uci.edu/data.php?id=11'> [1]</a>",  
    font= dict(size=12),
    showlegend=False,
    autosize=False,
    width=width,
    height=height,
    xaxis=layout.XAxis(axis),
    yaxis=layout.YAxis(axis),          
    margin=layout.Margin(
        l=40,
        r=40,
        b=85,
        t=100,
    ),
    hovermode='closest',
    annotations=[
           dict(
           showarrow=False, 
            text='This igraph.Graph has the Kamada-Kawai layout',  
            xref='paper',     
            yref='paper',     
            x=0,  
            y=-0.1,  
            xanchor='left',   
            yanchor='bottom',  
            font=dict(
            size=14 
            )     
            )
        ]          
    )

data=[trace1, trace2]
fig=Figure(data=data, layout=layout)
py.iplot(fig, filename='Coautorship-network-igraph') 

AttributeError: 'tuple' object has no attribute 'XAxis'